In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Struct Type Practice").getOrCreate()

In [2]:
from pyspark.sql.types import (StructType, StructField, StringType, IntegerType, LongType)

In [3]:
raw_data = [
    ("U001","Abhishek",28,"Hyderabad",50000),
    ("U002","Neha",32,"Delhi",62000),
    ("U003","Ravi",25,"Bangalore",45000),
    ("U004","Pooja",29,"Mumbai",58000)
]

user_schema = StructType([
    StructField("user_id", StringType(), nullable=False),
    StructField("name", StringType(), nullable=True),
    StructField("age", IntegerType(), nullable=True),
    StructField("city", StringType(), nullable=True),
    StructField("salary", IntegerType(), nullable=True),
])

df_users = spark.createDataFrame(raw_data, schema = user_schema)
df_users.printSchema()
df_users.show()

root
 |-- user_id: string (nullable = false)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- salary: integer (nullable = true)

+-------+--------+---+---------+------+
|user_id|    name|age|     city|salary|
+-------+--------+---+---------+------+
|   U001|Abhishek| 28|Hyderabad| 50000|
|   U002|    Neha| 32|    Delhi| 62000|
|   U003|    Ravi| 25|Bangalore| 45000|
|   U004|   Pooja| 29|   Mumbai| 58000|
+-------+--------+---+---------+------+



In [ ]:
#Wrong Data
raw_data2 = [(
    "U005","Amit","Thirty","Chennai",40000
)]
df_users = spark.createDataFrame(raw_data2, schema = user_schema)
df_users.show()

In [4]:
from pyspark.sql.types import ArrayType

interest_data = [
    ("U001",["AI","ML","Cloud"]),
    ("U002",["Testing","Automation"]),
    ("U003",["Data Engineering","Spark","Kafka"]),
    ("U004",["UI/UX"])
]

In [5]:
interest_schema = StructType([
    StructField("user_id", StringType(),False),
    StructField("interests", ArrayType(StringType()), nullable=True)
])

In [6]:
df_interests = spark.createDataFrame(interest_data, schema=interest_schema)
df_interests.printSchema()
df_interests.show(truncate = False)

root
 |-- user_id: string (nullable = false)
 |-- interests: array (nullable = true)
 |    |-- element: string (containsNull = true)

+-------+--------------------------------+
|user_id|interests                       |
+-------+--------------------------------+
|U001   |[AI, ML, Cloud]                 |
|U002   |[Testing, Automation]           |
|U003   |[Data Engineering, Spark, Kafka]|
|U004   |[UI/UX]                         |
+-------+--------------------------------+



In [7]:
from pyspark.sql.functions import explode

df_interests = df_interests.select("user_id", explode("interests").alias("interest")).show()

+-------+----------------+
|user_id|        interest|
+-------+----------------+
|   U001|              AI|
|   U001|              ML|
|   U001|           Cloud|
|   U002|         Testing|
|   U002|      Automation|
|   U003|Data Engineering|
|   U003|           Spark|
|   U003|           Kafka|
|   U004|           UI/UX|
+-------+----------------+



In [8]:
#Map Type
from pyspark.sql.types import MapType

In [9]:
device_data = [
    ("U001",{"mobile":120,"laptop":300}),
    ("U002",{"tablet":80}),
    ("U003",{"mobile":200,"desktop":400}),
    ("U004",{"laptop":250})
]

In [11]:
device_schema = StructType([
    StructField("user_id", StringType(), False),
    StructField("device_usuage", MapType(StringType(), IntegerType()), True)
])

df_devices = spark.createDataFrame(device_data, schema=device_schema)
df_devices.printSchema()
df_devices.show(truncate = False)

root
 |-- user_id: string (nullable = false)
 |-- device_usuage: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = true)

+-------+-------------------------------+
|user_id|device_usuage                  |
+-------+-------------------------------+
|U001   |{mobile -> 120, laptop -> 300} |
|U002   |{tablet -> 80}                 |
|U003   |{mobile -> 200, desktop -> 400}|
|U004   |{laptop -> 250}                |
+-------+-------------------------------+



In [12]:
nested_data = [
    ("U001",("Hyderabad","Telangana",500081)),
    ("U002",("Delhi","Delhi",110001)),
    ("U003",("Bangalore","Karnataka",560001))
]

In [13]:
address_schema = StructType([
    StructField("city", StringType(), True),
    StructField("state", StringType(), False),
    StructField("pincode",IntegerType(), True)
])

profile_schema = StructType([
    StructField("user_id", StringType(), False),
    StructField("address", address_schema, True)
])

In [15]:
df_profiles = spark.createDataFrame(nested_data, schema=profile_schema)
df_profiles.printSchema()
df_profiles.show(truncate=False)
#df_profiles.count()

root
 |-- user_id: string (nullable = false)
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- state: string (nullable = false)
 |    |-- pincode: integer (nullable = true)

+-------+------------------------------+
|user_id|address                       |
+-------+------------------------------+
|U001   |{Hyderabad, Telangana, 500081}|
|U002   |{Delhi, Delhi, 110001}        |
|U003   |{Bangalore, Karnataka, 560001}|
+-------+------------------------------+



3

In [16]:
df_profiles.select(
    "user_id",
    "address.city",
    "address.state",
).show()

+-------+---------+---------+
|user_id|     city|    state|
+-------+---------+---------+
|   U001|Hyderabad|Telangana|
|   U002|    Delhi|    Delhi|
|   U003|Bangalore|Karnataka|
+-------+---------+---------+



In [17]:
from pyspark.sql.functions import col

df_users.withColumn("salary_int", col("salary").cast("int")).show()

+-------+--------+---+---------+------+----------+
|user_id|    name|age|     city|salary|salary_int|
+-------+--------+---+---------+------+----------+
|   U001|Abhishek| 28|Hyderabad| 50000|     50000|
|   U002|    Neha| 32|    Delhi| 62000|     62000|
|   U003|    Ravi| 25|Bangalore| 45000|     45000|
|   U004|   Pooja| 29|   Mumbai| 58000|     58000|
+-------+--------+---+---------+------+----------+



In [18]:
from pyspark.sql.functions import to_date

df_orders.withColumn("order_date", to_date("order_date", "yyyy-MM-dd")).show()

NameError: name 'df_orders' is not defined